In [ ]:
"""
Author: Chunyu Yan
Date: 2025-07-15
"""

# Written by Chunyu
# This file is for testing /data/seanchoi/airflow/db_utils/push_sec_reports.py and /data/seanchoi/airflow/db_utils/search_db.py

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # .ipynb file
sys.path.append(project_root)

from collections import Counter
from datetime import datetime
from bson import ObjectId
from datetime import datetime
from db_utils import *

In [ ]:
company_name = "Amazon.com, Inc."
results_ticker = list(filter_files(COMPANY_TICKER_DATA, {"Company Name": company_name}))
results_ticker

# NVIDIA Corporation

In [ ]:
company_ticker_id = ObjectId("6765822dc2568cb07c25163e")  # 示例 company_ticker_id
start = datetime(2023, 1, 1)
end = datetime(2025, 12, 31)

quarter_reports = find_quarter_reports(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

annual_reports = find_annual_reports(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

transcripts = find_transcripts(
    company_ticker_id=company_ticker_id,
    start_date=start,
    end_date=end
)

In [ ]:
len(quarter_reports), len(annual_reports), len(transcripts)

In [ ]:
final_response = query_across_collections("Amazon.com, Inc.", "2022-01-01", "2026-01-01")
# ## to test news
# final_response = query_across_collections("Motors Liquidation Co", "2002-01-01", "2023-01-01")
# print(final_response)

In [ ]:
# save sec_reports to json file for better preview
import bson.json_util
import json, bson

save_file = "./final_response.json"
with open(save_file, "w", encoding="utf-8") as f:
    json.dump(final_response, f, default=bson.json_util.default, ensure_ascii=False, indent=2)

In [ ]:
# save sec_reports to json file for better preview
from db_utils import get_db, SEC_REPORT_COLLECTION, save_json

_, db = get_db()
collection = db[SEC_REPORT_COLLECTION]
cursor = collection.find().sort("_id", -1).limit(10)

save_file = "./sec_reports_sorted.json"
save_json(cursor, save_file)